# Character level language model - Dinosaurus land
### Keras Implementation of the Deep-Learning Specializiation project


Naming new dinos using RNNs

In [1]:
from keras.models import Model
from keras.layers import Input, SimpleRNN, Activation, Dense
from keras.utils import to_categorical
from keras.callbacks import LambdaCallback
import tensorflow as tf
import random
import sys

/Users/gerardoduran/anaconda/envs/deeplearning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
with open("dinos.txt", "r") as f:
    dinos = f.read().lower()
characters = sorted(list(set(dinos)))

In [3]:
np.random.choice(dinos.split(), 7)

array(['kelmayisaurus', 'cladeiodon', 'tylocephale', 'morinosaurus',
       'loricosaurus', 'viavenator', 'xenoceratops'], dtype='<U23')

In [4]:
# Map from all characters in the training dataset to a uniquex index
char_to_ix = {ix:char for char, ix in enumerate(characters)}
# Reverse map of char_to_ix to retrieve the index given the character
ix_to_char = {char:ix for char, ix in enumerate(characters,)}

nchars = len(characters)
nvocab = len(dinos)

In [5]:
char_to_ix

{'\n': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [6]:
# Size of the sequence to pass as training example
seqlen = 10
xtrain, ytrain = [], []
for i in range(0, nvocab - seqlen):
    xt = dinos[i: i + seqlen]
    yt = dinos[i + seqlen]
    xtrain.append([char_to_ix[char] for char in xt])
    ytrain.append([char_to_ix[char] for char in yt])
    
# training dataset is now
# of shape (xtrain X seqlen)
print(f"Number of training instances: {len(xtrain):,}")

# Reshaping into the form:
# nfeatures X timesteps X features
xtrain = np.reshape(xtrain, (-1, seqlen, 1))
# Normalizing values
#xtrain = xtrain / nvocab
# Transorming output values to be
# One-hot encoded
ytrain = to_categorical(ytrain)
xtrain.shape, ytrain.shape

Number of training instances: 19,903


((19903, 10, 1), (19903, 27))

In [ ]:
tf.reset_default_graph()
# Making RNN model with keras
xin = Input(shape=xtrain.shape[1:])
x = SimpleRNN(256, activation="tanh")(xin)
x = Dense(ytrain.shape[1], activation="softmax")(x)

model = Model(inputs=xin, outputs=x)
model.compile(optimizer="adam", loss="categorical_crossentropy")
model.fit(xtrain, ytrain, epochs=50)

Epoch 1/50
19903/19903 [==============================] - 3s 173us/step - loss: 2.4709
Epoch 2/50
19903/19903 [==============================] - 3s 166us/step - loss: 2.3002
Epoch 3/50
19903/19903 [==============================] - 3s 167us/step - loss: 2.2232
Epoch 4/50
19903/19903 [==============================] - 3s 166us/step - loss: 2.1577
Epoch 5/50
19903/19903 [==============================] - 3s 167us/step - loss: 2.0958
Epoch 6/50
19903/19903 [==============================] - 3s 167us/step - loss: 2.0438
Epoch 7/50
19903/19903 [==============================] - 3s 166us/step - loss: 1.9989
Epoch 8/50
19903/19903 [==============================] - 3s 170us/step - loss: 1.9612
Epoch 9/50
19903/19903 [==============================] - 4s 183us/step - loss: 1.9270
Epoch 10/50
19903/19903 [==============================] - 4s 182us/step - loss: 1.8943
Epoch 11/50
19903/19903 [==============================] - 4s 188us/step - loss: 1.8633
Epoch 12/50
19903/19903 [================

In [70]:
# pick a random seed
start = np.random.randint(0, len(xtrain)-1)
pattern = xtrain[start].ravel()
print("Seed:", end="")
print("\"", ''.join([ix_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = ix_to_char[index]
    seq_in = [ix_to_char[value] for value in pattern]
    print(result, end="")
    pattern = np.append(pattern, index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:" nacosaurus "

alasgataurus
laneansaurus
casanasaurus
alasgataurus
laneansaurus
casanasaurus
alasgataurus
laneansa
Done.


In [16]:
pattern.ravel()

array([0.])

In [ ]:
# Training dataset: Outputs
dinos_list = [[char_to_ix[c] for c in d + "\n"] for d in dinos.split()]
maxlen =  len(max(dinos_list, key=len))
ndinos = len(dinos_list)
nchars = len(characters)
xtrain = np.zeros((ndinos, maxlen, nchars))
ytrain = np.zeros((ndinos, maxlen, nchars))

xtrain = np.zeros((ndinos, maxlen))


In [ ]:
tf.reset_default_graph()
xin = Input(shape=(maxlen,))
x = SimpleRNN(10, activation="tanh")(xin)
x = Dense()
x = Activation("softmax")(x)

model = Model(inputs=xin, outputs=x)
model.compile("adam", loss="categorical_crossentropy")
model.fit(xtrain, ytrain)

In [16]:
model.predict(np.zeros((1, maxlen, nchars)))[0]

array([0.03703704, 0.03703704, 0.03703704, 0.03703704, 0.03703704,
       0.03703704, 0.03703704, 0.03703704, 0.03703704, 0.03703704,
       0.03703704, 0.03703704, 0.03703704, 0.03703704, 0.03703704,
       0.03703704, 0.03703704, 0.03703704, 0.03703704, 0.03703704,
       0.03703704, 0.03703704, 0.03703704, 0.03703704, 0.03703704,
       0.03703704, 0.03703704], dtype=float32)

In [18]:
np.zeros((1, maxlen, nchars))[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],


In [109]:
for ix, samp in enumerate(dinos_list):
    print(ix, samp)

0 [2, 2, 4, 9, 6, 15, 16, 20, 2, 22, 19, 22, 20, 1]
1 [2, 2, 19, 5, 16, 15, 26, 25, 1]
2 [2, 3, 5, 2, 13, 13, 2, 9, 20, 2, 22, 19, 22, 20, 1]
3 [2, 3, 6, 13, 10, 20, 2, 22, 19, 22, 20, 1]
4 [2, 3, 19, 10, 4, 21, 16, 20, 2, 22, 19, 22, 20, 1]
5 [2, 3, 19, 16, 20, 2, 22, 19, 22, 20, 1]
6 [2, 3, 26, 5, 16, 20, 2, 22, 19, 22, 20, 1]
7 [2, 4, 2, 15, 21, 9, 16, 17, 9, 16, 13, 10, 20, 1]
8 [2, 4, 9, 6, 13, 16, 22, 20, 2, 22, 19, 22, 20, 1]
9 [2, 4, 9, 6, 19, 16, 19, 2, 17, 21, 16, 19, 1]
10 [2, 4, 9, 10, 13, 13, 6, 20, 2, 22, 19, 22, 20, 1]
11 [2, 4, 9, 10, 13, 13, 16, 3, 2, 21, 16, 19, 1]
12 [2, 4, 19, 10, 20, 21, 2, 23, 22, 20, 1]
13 [2, 4, 19, 16, 4, 2, 15, 21, 9, 16, 20, 2, 22, 19, 22, 20, 1]
14 [2, 4, 19, 16, 21, 9, 16, 13, 22, 20, 1]
15 [2, 4, 21, 10, 16, 20, 2, 22, 19, 22, 20, 1]
16 [2, 5, 2, 14, 2, 15, 21, 10, 20, 2, 22, 19, 22, 20, 1]
17 [2, 5, 2, 20, 2, 22, 19, 22, 20, 1]
18 [2, 5, 6, 13, 16, 13, 16, 17, 9, 22, 20, 1]
19 [2, 5, 6, 16, 17, 2, 17, 17, 16, 20, 2, 22, 19, 22, 20, 1]
20 